Convert AlborEx positions data to JSON file easily displayed by  [Leaflet](https://leafletjs.com/).    
The functions defined is this notebook will be added as methods for the different classes of `alborexdata.py`.  

In [1]:
import alborexdata
import os
import geojson
import numpy as np
import netCDF4
import datetime
import json

/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [6]:
with open('alborexconfig.json') as json_data_file:
    config = json.load(json_data_file)
outputdir = config["datadirleaflet"]

# CTD

In [7]:
lonCTD1, latCTD1 = alborexdata.load_lonloat_ctdleg(config["datafiles"]["ctdlegs"][0])
lonCTD2, latCTD2 = alborexdata.load_lonloat_ctdleg(config["datafiles"]["ctdlegs"][1])

In [8]:
CTDlist = [[lat, lon] for lat, lon in zip(latCTD1, lonCTD1)]

In [9]:
with open(os.path.join(outputdir, 'CTD1.js'), 'w') as f:
    f.write("var CTD1list = " + str(CTDlist))

In [10]:
CTDlist = [[lat, lon] for lat, lon in zip(latCTD2, lonCTD2)]

In [11]:
with open(os.path.join(outputdir, 'CTD2.js'), 'w') as f:
    f.write("var CTD2list = " + str(CTDlist))

# Glider

In [13]:
reload(alborexdata)

<module 'alborexdata' from '/home/ctroupin/Publis/AlborEx-Data-Python/python/alborexdata.py'>

In [15]:
glider1 = alborexdata.Glider() 
glider2 = alborexdata.Glider() 

In [16]:
glider1.get_coords(config["datafiles"]["gliders"][0])
glider2.get_coords(config["datafiles"]["gliders"][1])

OSError: NetCDF: I/O failure

In [77]:
makeGeoJson(lon1, lat1, os.path.join(outputdir, 'Glider1.js'), 'glider1', NN=100)

In [78]:
makeGeoJson(lon2, lat2, os.path.join(outputdir, 'Glider2.js'), 'glider2', NN=100)

# Drifters

We've created a list of deployments (text file) using [get_deployments_alborex](get_deployments_alborex.ipynb).

In [6]:
drifterfilelist = './deploymentList.txt'

Dates of interest for the extraction

In [56]:
dateinit = datetime.datetime(2014, 5, 25)
dateend = datetime.datetime(2014, 6, 15)
timeinit = (dateinit-datetime.datetime(1970,1,1)).total_seconds()
timeend = (dateend-datetime.datetime(1970,1,1)).total_seconds()

In [66]:
lonall, latall = [], []
with open(drifterfilelist, 'r') as f:
    for drifterfile in f:
        with netCDF4.Dataset(drifterfile) as nc:
            time = nc.variables['time'][:]
            timeunits = nc.variables['time'].units
            goodtime = np.where(np.logical_and((time > timeinit),(time <= timeend)))[0]
            
            # Only take values in the considered period
            lon = nc.variables['LON'][goodtime]
            lat = nc.variables['LAT'][goodtime]
            QClon = nc.variables['QC_LON'][goodtime]
            QClat = nc.variables['QC_LAT'][goodtime]
            # Only take good coordinates
            goodcoords = np.where(np.logical_and((QClon==1),(QClat==1)))[0]
            lon = lon[goodcoords]
            lat = lat[goodcoords]
            
            lonall.append(lon.tolist())
            latall.append(lat.tolist())
            #print(lon.min(), lon.max(), lat.min(), lat.max())

In [67]:
# Create list of tuples
DrifterList = [[(lon, lat) for lon, lat in zip(londep, latdep)] for londep, latdep in zip(lonall, latall)]

In [68]:
DrifterGeojson = geojson.MultiLineString(DrifterList)  

In [69]:
with open(os.path.join(outputdir, 'drifters.js'), 'w') as f:
    f.write("var drifters = ")
    geojson.dump(DrifterGeojson, f)

# Profilers

Same as drifters.

In [77]:
profilerfilelist = './alborexProfilerList.txt'

In [99]:
lonall, latall = [], []
with open(profilerfilelist, 'r') as f:
    for profilerfile in f:
        with netCDF4.Dataset(profilerfile) as nc:
            time = nc.variables['time'][:]
            timeunits = nc.variables['time'].units
            goodtime = np.where(np.logical_and((time > timeinit),(time <= timeend)))[0]
            
            # Only take values in the considered period
            lon = nc.variables['LON'][goodtime[1:]]
            lat = nc.variables['LAT'][goodtime[1:]]
            """
            QClon = nc.variables['QC_LON'][goodtime]
            QClat = nc.variables['QC_LAT'][goodtime]
            # Only take good coordinates
            goodcoords = np.where(np.logical_and((QClon==1),(QClat==1)))[0]
            lon = lon[goodcoords]
            lat = lat[goodcoords]
            """
            lonall.append(lon.tolist())
            latall.append(lat.tolist())
            #print(lon.min(), lon.max(), lat.min(), lat.max())

In [100]:
# Create list of tuples
ProfilerList = [[(lon, lat) for lon, lat in zip(londep, latdep)] for londep, latdep in zip(lonall, latall)]
ProfilerGeojson = geojson.MultiLineString(ProfilerList)  
with open(os.path.join(outputdir, 'profilers.js'), 'w') as f:
    f.write("var profilers = ")
    geojson.dump(ProfilerGeojson, f)